In [1]:
import re
import numpy as np
import pandas as pd
import pickle
import string
# nltk
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
#Keras
import keras
from keras.preprocessing import sequence

In [2]:
load_model=keras.models.load_model("hate&abusive_model.h5")
with open('tokenizer.pickle', 'rb') as handle:
    load_tokenizer = pickle.load(handle)

In [3]:
test=pd.read_csv('tweets.csv', header=None, names=['tweet'])
#test.rename(columns ={'tweet'}, inplace = True)
test.head(5)

,tweet
0,"A96 A90 Aberdeen - A92 Aberdeen - Closure, All..."
1,"G'night, that's me signing off. We'll be back ..."
2,"A1 - Tyne Bridge - Weather, All lanes restrict..."
3,"Clackmannanshire - Weather, Drivers in Clackma..."
4,"West Lothian - Weather, Drivers in West Lothia..."


In [4]:
stopwordlist = ['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'but', 'by', 'can', 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from',
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
             'into','is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're','s', 'same', 'she', "shes", 'should', "shouldve",'so', 'some', 'such',
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those',
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'won', 'y', 'you','youu','yous', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves']

STOPWORDS = set(stopwordlist)

In [5]:
test=pd.read_csv('tweets.csv', header=None, names=['tweet'])
#test.rename(columns ={'tweet'}, inplace = True)
test.head(5)

test0 = test1 = test

english_punctuations = string.punctuation
st = nltk.PorterStemmer()
lm = nltk.WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'w+')

def cleaning_data(data):
    
    data=data.lower().strip()
    #Removing everything other than a-z
    data = re.sub('[^a-z\\s]+', '', data)
         
    #Removing punctuations
    translator = str.maketrans('', '', english_punctuations)
    data = data.translate(translator)
    
    #Removing repeating characters
    data = re.sub(r'(.)1+', r'1', data)
    
    #Removing URLs
    data = re.sub('((www.[^s]+)|(https?://[^s]+))',' ',data)
    
    #Removing stopwords and tokenizing
    d=[]
    words=data.split(" ")
    for w in words:
        if w not in STOPWORDS:
            d.append(w)
    data=d
    
    #Stemming and Lemmatize
    data = [st.stem(word) for word in data]
    data = [lm.lemmatize(word) for word in data]
    return data

test0['tweet'] = test['tweet'].apply(lambda text: cleaning_data(text))
test0.head(10)

,tweet
0,"[aberdeen, , aberdeen, , closur, lane, close, ..."
1,"[gnight, that, sign, off, well, back, , tomorr..."
2,"[, tyne, bridg, , weather, lane, restrict, dir..."
3,"[clackmannanshir, , weather, driver, clackmann..."
4,"[west, lothian, , weather, driver, west, lothi..."
5,"[falkirk, , weather, driver, falkirk, advis, u..."
6,"[stirl, , weather, driver, stirl, advis, use, ..."
7,"[got, somewher, need, unsur, road, are\r\n\r\n..."
8,"[south, lanarkshir, , weather, driver, south, ..."
9,"[north, lanarkshir, , weather, driver, north, ..."


In [9]:
seq = load_tokenizer.texts_to_sequences(test0['tweet'])
padded = sequence.pad_sequences(seq, maxlen=300)
print("seq",seq)
pred =[]
hate=0
no_hate=0
pred = load_model.predict(padded)
print("pred", pred)
for p in pred:
    if p<0.5:
        no_hate+=1
        print("no hate")
    else:
        hate+=1
        print("hate and abusive")


seq [[16391, 16391, 4979, 824, 2138], [14, 1305, 114, 191, 109, 262], [990, 4979, 723], [990, 2996, 376, 22017, 1718, 691, 6689, 937], [2145, 990, 2996, 2145, 376, 22017, 1718, 691, 6689, 937], [990, 2996, 376, 22017, 1718, 691, 6689, 937], [990, 2996, 376, 22017, 1718, 691, 6689, 937], [56, 106, 1465, 250, 3022, 2702, 1362, 717, 44, 3088, 34208, 8114], [1510, 990, 2996, 1510, 376, 22017, 1718, 691, 6689, 937], [5040, 990, 2996, 5040, 376, 22017, 1718, 691, 6689, 937], [187, 284, 106, 422, 1423, 223, 7058, 187, 101, 1576, 274, 187], [8, 23505, 1182, 824, 647, 5040, 623, 1845, 677, 2500, 1266], [1142, 4945, 630, 173, 1372, 4595, 2500, 87, 173, 80, 949, 465, 1142, 842, 1129], [16391, 16391, 4979, 824], [308, 4979, 723, 2138], [34208, 105, 101, 2492], [791, 291, 857, 4979, 824], [174, 78, 2664, 106, 18924, 36809, 223, 2036, 78, 2025, 2222, 9957, 422, 574], [14, 1498, 12809, 211, 198, 417], [4979, 824, 38950, 2138], [791, 1465, 4979, 824], [650, 20, 376, 436, 5606, 388, 937, 15349, 100, 43

In [12]:
print("Total number of hate tweets:",hate)
print("Total number of non-hate tweets:",no_hate)
print("Hate & Abusive :",(hate/(hate+no_hate))*100,"%")
print("No Hate :",(no_hate/(hate+no_hate))*100,"%")

Total number of hate tweets: 8
Total number of non-hate tweets: 49
Hate & Abusive : 14.035087719298245 %
No Hate : 85.96491228070175 %
